# <h1 style="color:rgb(228, 12, 33); text-align: center;">De la teoría a la práctica: LSTM y Transformers en PyTorch</h1>

---
![imagen.png](https://discuss.pytorch.org/uploads/default/6415da0424dd66f2f5b134709b92baa59e604c55)

<div style="background-color: rgba(100, 108, 116, 0.1); padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <p>Bienvenido a este cuaderno de Kaggle, donde profundizaremos en la comprensión e implementación de redes de memoria a corto plazo (LSTM) usando PyTorch, un marco de aprendizaje profundo poderoso. Pero antes de profundizar en las complejidades de LSTM, dediquemos un momento a comprender los conceptos básicos de datos de series temporales, redes neuronales recurrentes (RNN) y LSTM.</p>
     <h2 style="color:rgb(31, 103, 211);">Datos de series temporales</h2>
     <p>Los datos de series temporales son una secuencia de puntos de datos numéricos tomados en puntos sucesivos igualmente espaciados en el tiempo. Estos puntos de datos están ordenados y dependen de los puntos de datos anteriores, lo que hace que los datos de series temporales sean los principales candidatos para las predicciones. Los ejemplos de datos de series temporales incluyen precios de acciones, previsiones meteorológicas y datos de ventas, entre muchos otros.</p>
     <h2 style="color:rgb(31, 103, 211);">Redes neuronales recurrentes (RNN)</h2>
     <p>Las redes neuronales tradicionales tienen problemas con los datos de series temporales debido a su incapacidad para recordar entradas anteriores en su estado actual. Sin embargo, las redes neuronales recurrentes (RNN) están diseñadas para abordar este problema. Las RNN son una clase de redes neuronales artificiales donde las conexiones entre nodos forman un gráfico dirigido a lo largo de una secuencia temporal. Esto les permite usar su estado interno (memoria) para procesar secuencias de entradas, haciéndolos ideales para datos dependientes del tiempo.</p>
     <p>Sin embargo, los RNN sufren ciertas limitaciones. Tienen dificultades para manejar las dependencias a largo plazo debido al problema del "gradiente de fuga", donde la contribución de la información decae geométricamente con el tiempo, lo que dificulta que la RNN aprenda de las capas anteriores.</p>
     <h2 style="color:rgb(31, 103, 211);">Memoria a corto plazo (LSTM)</h2>
     <p>Las redes de memoria a largo plazo o LSTM son un tipo especial de RNN capaz de aprender dependencias a largo plazo. Presentados por Hochreiter y Schmidhuber en 1997, los LSTM tienen un diseño único que ayuda a combatir el problema del gradiente de fuga. Contienen un estado de celda y tres puertas (entrada, olvido y salida) para controlar el flujo de información dentro de la red, lo que les permite recordar u olvidar información durante largos períodos de tiempo.</p>
     <p>En este cuaderno, exploraremos cómo implementar correctamente LSTM en PyTorch y usarlo para tareas de predicción de series temporales. Cubriremos todo, desde los conceptos básicos de LSTM hasta su implementación, con el objetivo de proporcionar una comprensión integral de esta poderosa arquitectura de red neuronal. ¡Empecemos!</p>
</div>


<div style="background-color: rgba(100, 108, 116, 0.1); padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h2 style="color:rgb(31, 103, 211);">Comprender la entrada y salida en torch.nn.RNN</h2>
     <p>En esta sección, profundizaremos en los detalles de los parámetros de entrada y salida del módulo torch.nn.RNN, una implementación de red neuronal recurrente (RNN) integrada en la biblioteca PyTorch. Es crucial comprender estos parámetros para aprovechar al máximo las capacidades de RNN de PyTorch en nuestra implementación de LSTM.</p>
     <h3 style="color:rgb(172, 28, 44);">Entrada a torch.nn.RNN</h3>
     <p>El módulo torch.nn.RNN acepta dos entradas principales:</p>
     <ul>
         <li><b>entrada</b>: Esto representa la secuencia que se alimenta a la red. El tamaño esperado es (seq_len, lote, input_size). Sin embargo, si se especifica batch_first=True, entonces el tamaño de entrada debe reorganizarse a (batch, seq_len, input_size).</li>
         <li><b>h_0</b>: representa el estado oculto inicial de la red en el paso de tiempo t=0. Por defecto, si no inicializamos esta capa oculta, PyTorch la inicializará automáticamente con ceros. El tamaño de h_0 debe ser (num_layers * num_directions, batch, input_size), donde num_layers representa la cantidad de RNN apilados y num_directions es igual a 2 para RNN bidireccionales y 1 en caso contrario.</li>
     </ul>
     <h3 style="color:rgb(172, 28, 44);">Salida de torch.nn.RNN</h3>
     <p>El módulo torch.nn.RNN proporciona dos salidas:</p>
     <ul>
         <li><b>out</b>: esto representa la salida de la última capa RNN para todos los pasos de tiempo. El tamaño es (seq_len, lote, num_directions * hidden_size). Sin embargo, si se especifica batch_first=True, el tamaño de salida se convierte en (batch, seq_len, num_directions * hidden_size).</li>
         <li><b>h_n</b>: este es el valor de estado oculto del último paso de tiempo en todas las capas RNN. El tamaño es (num_layers * num_directions, lote, hidden_size). A diferencia de la entrada, el h_n no se ve afectado por batch_first=True.</li>
     </ul>
     <p>Para visualizar mejor estas entradas y salidas, consulte el siguiente diagrama. En este caso, asumimos un tamaño de lote de 1. Si bien el diagrama ilustra un LSTM, que tiene dos parámetros ocultos (h, c), tenga en cuenta que RNN y GRU solo tienen h.</p>
     <p>Al comprender estos parámetros, podemos aprovechar el poder del módulo torch.nn.RNN y crear modelos efectivos para nuestros datos de series temporales utilizando LSTM. Continuemos nuestra exploración de LSTM con PyTorch en las siguientes secciones.</p>
</div>

![image.png](https://miro.medium.com/max/576/1*tUxl5-C-t3Qumt0cyVhm2g.png)

<a id="TdC"></a>
# Tabla de contenido
- [1. Importaciones](#1)
- [2. LSTM](#2)
     - [Muchos a uno] (# 2.1)
     - [Muchos a muchos] (# 2.2)
     - [Secuencia de generación de muchos a muchos] (# 2.3)
- [3. Transformadores] (#3)
     - [Entrada de enmascaramiento](#3.1)
     - [fichas SOS y EOS] (#3.2)

<a id="1"></a>
# **<div style="padding:10px;color:white;display:fill;border-radius:5px;background-color:rgb(31, 103, 211);font-size:120%;font-family: Verdana;"><center><span> Importaciones </span></center></div>**

In [1]:
import torch 
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset

from tqdm import tqdm 
import numpy as np 
import pandas as pd 
import random
import matplotlib.pyplot as plt 
import seaborn as sns
sns.set_style('white')

/opt/conda/lib/python3.10/site-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.5
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h2 style="color:rgb(31, 103, 211);">Acerca del conjunto de datos</h2>
     <p>En este cuaderno, utilizaremos una serie de datos de tiempo simple para probar y comprender la aplicación de los modelos LSTM y Transformer. El conjunto de datos elegido es bastante sencillo: un rango de números que comienza en 0 y termina en 1000. Esta simplicidad nos permitirá centrarnos más en el funcionamiento de los modelos LSTM y Transformer, examinando qué tan bien pueden comprender y procesar datos numéricos secuenciales simples. . A través de esto, nuestro objetivo es lograr una comprensión clara de estas poderosas técnicas de aprendizaje profundo.</p>
</div>

<a id="2"></a>
# **<div style="padding:10px;color:white;display:fill;border-radius:5px;background-color:rgb(31, 103, 211);font-size:120%;font-family:Verdana;"><center><span> LSTM </span></center></div>**

<aid="2.1"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h2 style="color:rgb(31, 103, 211);">Comprensión de la arquitectura muchos a uno en LSTM</h2>
     <p>Las redes de memoria a corto plazo (LSTM), como todas las redes neuronales recurrentes (RNN), son reconocidas por su capacidad para procesar datos secuenciales. Uno de los aspectos clave que los hace flexibles y potentes son los diversos tipos de arquitecturas de entrada y salida que pueden adoptar, una de las cuales es la arquitectura Muchos a Uno.</p>
     <p>En una arquitectura LSTM de muchos a uno, el modelo acepta una secuencia de entradas en varios pasos de tiempo y produce una sola salida. En cada paso de tiempo, la celda LSTM toma una entrada y el estado oculto de la celda anterior, los procesa y pasa su propio estado oculto a la siguiente celda.</p>
     <p>A pesar de recibir información en cada paso de tiempo, el LSTM de muchos a uno solo produce su salida final en el último paso de tiempo. Esta característica hace que las redes LSTM de muchos a uno sean particularmente útiles para tareas como el análisis de opiniones, donde un modelo lee una secuencia de palabras (entrada) y genera una única puntuación de opinión, o clasificación de texto, donde un documento se lee secuencialmente y una sola clase se emite la etiqueta.</p>
     <p>A través del poder de LSTM y la flexibilidad de arquitecturas como Many-to-One, podemos abordar de manera efectiva una amplia gama de problemas basados en secuencias en el mundo del aprendizaje automático y la inteligencia artificial.</p>
</div>


### Crear cargador de datos personalizado [multinúcleo]

In [3]:
class CustomDataset(Dataset):
    def __init__(self, seq_len=5, max_len=1000):
        super(CustomDataset).__init__()
        self.datalist = np.arange(0,max_len)
        self.data, self.targets = self.timeseries(self.datalist, seq_len)
        
    def __len__(self):
        return len(self.data)
    
    def timeseries(self, data, window):
        temp = []
        targ = data[window:]
        for i in range(len(data)-window):
            temp.append(data[i:i+window])

        return np.array(temp), targ
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x,y
    
dataset = CustomDataset(seq_len=5, max_len=1000)

In [4]:
for x,y in dataset:
    print(x,y)
    break

tensor([0., 1., 2., 3., 4.]) tensor(5.)


In [6]:
dataloader = DataLoader(dataset, batch_size=4, shuffle=True, num_workers=4)
#collate_fn=custom_collector

In [7]:
for x,y in dataloader:
    print(x,y)
    break

tensor([[378., 379., 380., 381., 382.],
        [949., 950., 951., 952., 953.],
        [498., 499., 500., 501., 502.],
        [446., 447., 448., 449., 450.]]) tensor([383., 954., 503., 451.])


<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
<p>Echemos un vistazo más de cerca a nuestro caso de uso específico para la arquitectura LSTM de muchos a uno. En nuestro escenario, estamos alimentando el LSTM con una secuencia de 5 números aleatorios y anticipamos que el modelo predecirá el sexto número de la secuencia. Si bien hemos elegido una serie sencilla de números incrementales para este ejemplo, las posibles aplicaciones de este concepto se extienden mucho más.</p>

<p style="color:rgb(172, 28, 44);">Imagínese que esta secuencia es una serie temporal de datos de precios de acciones, condiciones climáticas o incluso una serie de pasos en una pregunta de razonamiento lógico. La capacidad de predecir el próximo evento en función de una serie de eventos anteriores es un aspecto crítico en muchos campos, incluidas las finanzas, la meteorología y la inteligencia artificial. Al entrenar nuestro modelo LSTM para comprender y predecir estas secuencias, podemos aprovechar la arquitectura LSTM de muchos a uno para resolver problemas complejos en estas áreas y más allá.</p>

</div>

In [10]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers):
        super().__init__()
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # estados ocultos no definidos por lo tanto el valor de h0,c0 == (0,0)
        out, (hn, cn) = self.lstm(x)
        
        # como sugiere el diagrama para tomar la última salida en muchos a uno
        # imprimir(out.shape)
        # imprimir(hn.shape)
        # todo lote, última columna de secuencia, todos los valores ocultos
        out = out[:, -1, :]
        out = self.fc(out)
        
        return out

In [13]:
model = RNN(input_size=1, hidden_size=256, num_layers=2)

In [14]:
t = torch.tensor([11,12,13,14,15]).type(torch.Tensor).view(1,-1,1)
t.shape

torch.Size([1, 5, 1])

In [15]:
model(t)

tensor([[0.0238]], grad_fn=<AddmmBackward0>)

### Training 

In [17]:
loss_function = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [18]:
for e in tqdm(range(50)):
    i = 0
    for x,y in dataloader:
        optimizer.zero_grad()

        x = torch.unsqueeze(x, 0).permute(1,2,0)
        # forward
        # adelante
        predictions = model(x)

        loss = loss_function(predictions.view(-1), y)
        
        # backward
        # al revés
        loss.backward()

        # optimization
        optimizer.step()

        i+=1
    if e%5==0:
        print(loss.detach().numpy())

  2%|▏         | 1/50 [00:03<02:46,  3.40s/it]

598389.2


 12%|█▏        | 6/50 [00:19<02:16,  3.11s/it]

429288.66


 22%|██▏       | 11/50 [00:34<02:00,  3.08s/it]

...

 82%|████████▏ | 41/50 [02:09<00:28,  3.14s/it]

32.080643


 92%|█████████▏| 46/50 [02:25<00:12,  3.16s/it]

314.70874


100%|██████████| 50/50 [02:38<00:00,  3.17s/it]


In [21]:
input_tensor = torch.tensor([10,11,12,13,14,15,16,17]).type(torch.Tensor).view(1,-1,1)
model(input_tensor)

tensor([[17.6299]], grad_fn=<AddmmBackward0>)

<aid="2.2"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h2 style="color:rgb(31, 103, 211);">Comprensión de la arquitectura de muchos a muchos en LSTM</h2>
    <p>Otra arquitectura crucial en el mundo de las redes de memoria a corto plazo (LSTM), un tipo de red neuronal recurrente (RNN), es la arquitectura de muchos a muchos. Esta arquitectura ofrece una forma versátil de manejar un conjunto diverso de problemas que involucran datos secuenciales.</p>
     <p>En una arquitectura LSTM de muchos a muchos, el modelo procesa una secuencia de entradas en varios pasos de tiempo y genera una secuencia de salidas. En esta configuración, cada celda LSTM toma una entrada y el estado oculto de la celda anterior en cada paso de tiempo, luego produce una salida junto con su propio estado oculto que pasa a la siguiente celda.</p>
     <p>A diferencia del LSTM de muchos a uno, el LSTM de muchos a muchos no espera hasta el último paso de tiempo para producir una salida. En su lugar, genera una salida en cada paso de tiempo. Esto hace que las redes LSTM de muchos a muchos sean muy útiles para tareas como la traducción automática, donde una secuencia de palabras en un idioma (entrada) se traduce a una secuencia de palabras en otro idioma (salida).</p>
     <p>La arquitectura Many-to-Many de LSTM abre una amplia gama de posibilidades, lo que la convierte en una poderosa herramienta en los ámbitos del aprendizaje automático y la inteligencia artificial.</p>
</div>

In [22]:
class CustomDataset(Dataset):
    def __init__(self, seq_len=50, future=5,  max_len=1000):
        super(CustomDataset).__init__()
        self.datalist = np.arange(0,max_len)
        self.data, self.targets = self.timeseries(self.datalist, seq_len, future)
        
    def __len__(self):
        #this len will decide the index range in getitem
        return len(self.targets)
    
    def timeseries(self, data, window, future):
        temp = []
        targ = []
        
        for i in range(len(data)-window):
            temp.append(data[i:i+window])
            
        for i in range(len(data)-window -future):
            targ.append(data[i+window:i+window+future])

        return np.array(temp), targ
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x,y
    
dataset = CustomDataset(seq_len=50, future=5, max_len=1000)

In [23]:
for x,y in dataset:
    print(x.shape, y.shape)
    break

torch.Size([50]) torch.Size([5])


In [24]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)
#collate_fn=custom_collector

In [25]:
for x,y in dataloader:
    print(x.shape, y.shape)
    break

torch.Size([8, 50]) torch.Size([8, 5])


In [26]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, future=5):
        super().__init__()
        self.future = future
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, future)

    def forward(self, x):
        # hidden states not defnined hence the value of h0,c0 == (0,0)
        out, (hn, cn) = self.lstm(x)
        
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, future=5):
        super().__init__()
        self.future = future
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, future)

    def forward(self, x):
        # hidden states not defnined hence the value of h0,c0 == (0,0)
        out, (hn, cn) = self.lstm(x)
        
        # as the diagram suggest to take the last output in many to one 
        # print(out.shape) 
        # print(hn.shape)
        # all batch, last column of seq, all hidden values
        out = out[:, -self.future, :]
        out = self.fc(out)
        
        return out

In [27]:
model = RNN(input_size=1, hidden_size=256, num_layers=2, future=5)

In [28]:
d = 45
t = torch.tensor(np.arange(d,d+50)).type(torch.Tensor).view(1,-1,1)
t.shape

torch.Size([1, 50, 1])

In [29]:
model(t)

tensor([[-0.1343, -0.0079,  0.0052, -0.0008, -0.0501]],
       grad_fn=<AddmmBackward0>)

In [30]:
loss_function = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for e in tqdm(range(50)):
    i = 0
    avg_loss = []
    for x,y in dataloader:
        optimizer.zero_grad()

        x = torch.unsqueeze(x, 0).permute(1,2,0)
        # forward
        predictions = model(x)
        
        # loss
        loss = loss_function(predictions, y)
        
        # backward
        loss.backward()

        # optimization
        optimizer.step()
        avg_loss.append(loss.detach().numpy())

        i+=1
    if e%2==0:
        avg_loss = np.array(avg_loss)
        print(avg_loss.mean())

<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
<p>Después de introducir los 50 términos iniciales de nuestra secuencia en el modelo, comenzamos a observar algunos resultados prometedores. Parece que el modelo está aprendiendo con éxito a reconocer los patrones subyacentes en la secuencia.</p>
<p>El resultado generado por el modelo parece adherirse a la lógica de la secuencia, lo que sugiere que la arquitectura LSTM está capturando y comprendiendo de manera efectiva las dependencias secuenciales. Esta capacidad de discernir patrones y extrapolarlos es un aspecto poderoso de las redes LSTM, y es gratificante verlo funcionar en nuestro modelo.</p>
<p>Estos primeros resultados son alentadores e indican que nuestro modelo va por buen camino. A medida que continuamos refinando y entrenando nuestro LSTM, podemos esperar que se vuelva aún más experto en comprender y predecir la secuencia.</p>
</div>

In [ ]:
d = random.randint(0,1000)
t = torch.tensor(np.arange(d,d+50)).type(torch.Tensor).view(1,-1,1)
r = model(t).view(-1)

In [ ]:
fig = plt.figure(figsize=(16,4))
plt_x = np.arange(0,t.shape[1]+len(r))
plt_y = np.arange(d,d+50+len(r))

plt_xp = np.arange(t.shape[1], t.shape[1]+len(r))
plt_yp = r.detach().numpy()
for i in range(len(r)):
    plt.scatter(plt_x, plt_y)
    plt.scatter(plt_xp, plt_yp)
    

<aid="2.3"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h2 style="color:rgb(31, 103, 211);">Comprensión de la generación de secuencias de muchos a muchos con LSTM</h2>
     <p>Cuando se trabaja con redes de memoria a corto plazo (LSTM), es fundamental comprender cómo se gestiona la generación de secuencias, especialmente en una configuración de muchos a muchos. En una arquitectura de este tipo, la salida de cada celda LSTM se puede utilizar como entrada para una red de realimentación posterior para generar una secuencia de salidas.</p>
     <p>Consideremos el siguiente bloque de código como ejemplo:</p>
     <pre style="background-color: #e0e0e0; padding: 10px; border-radius: 10px;">
         fuera, (hn, cn) = self.lstm(x)
         res = antorcha.ceros((fuera.forma[0], fuera.forma[1]))
         para b en el rango (out.shape[0]):
             alimentar = salir[b, :, :]
             _out = self.fc(feed).vista(-1)
             res[b] = _fuera
     </pre>
     <p>En este código, <code>self.lstm(x)</code> aplica la capa LSTM a la entrada <code>x</code>, generando una salida <code>out</code> y la final oculto y estados de celda <code>hn</code> y <code>cn</code>. Luego inicializamos un tensor de ceros <code>res</code> del mismo tamaño que <code>out</code> para almacenar nuestros resultados.</p>
     <p>Luego, para cada secuencia en la salida <code>out</code>, alimentamos la secuencia a través de una capa totalmente conectada <code>self.fc(feed)</code> y remodelamos la salida para que coincida con lo esperado. dimensiones usando <code>.view(-1)</code>. El resultado se almacena en la posición correspondiente en <code>res</code>.</p>
     <p>Este proceso ejemplifica cómo se puede usar una red LSTM de muchos a muchos para generar una secuencia de salidas, con la capa LSTM y una capa de avance posterior trabajando en conjunto para transformar una secuencia de entradas en una secuencia correspondiente de salidas.</p>
</div>


In [ ]:
class CustomDataset(Dataset):
    def __init__(self, seq_len=50, future=50,  max_len=1000):
        super(CustomDataset).__init__()
        self.datalist = np.arange(0,max_len)
        self.data, self.targets = self.timeseries(self.datalist, seq_len, future)
        
    def __len__(self):
        #this len will decide the index range in getitem
        return len(self.targets)
    
    def timeseries(self, data, window, future):
        temp = []
        targ = []
        
        for i in range(len(data)-window):
            temp.append(data[i:i+window])
            
        for i in range(len(data)-window -future):
            targ.append(data[i+future:i+window+future])

        return np.array(temp), targ
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        return x,y
    
dataset = CustomDataset(seq_len=50, future=5, max_len=1000)

In [ ]:
for x, y in dataset:
    print(x.shape, y.shape)
    break

In [ ]:
x

In [ ]:
y

In [ ]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)
#collate_fn=custom_collector

In [ ]:
class RNN(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, future=5):
        super().__init__()
        self.future = future
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        
        self.lstm = nn.LSTM(input_size,hidden_size,num_layers,batch_first=True)
        self.fc = nn.Linear(hidden_size, 1)

    def forward(self, x):
        # hidden states not defnined hence the value of h0,c0 == (0,0)
        out, (hn, cn) = self.lstm(x)
        
        # as the diagram suggest to take the last output in many to one 
        # print(out.shape)
        # print(hn.shape)
        # all batch, last column of seq, all hidden values
        res = torch.zeros((out.shape[0], out.shape[1]))
        for b in range(out.shape[0]):
            feed = out[b, :, :]
            _out = self.fc(feed).view(-1)
            res[b] = _out
        
        return res

In [ ]:
model = RNN(input_size=1, hidden_size=256, num_layers=2, future=5)

In [ ]:
t = torch.tensor(np.arange(d,d+50)).type(torch.Tensor).view(1,-1,1)
r = model(t).view(-1)

In [ ]:
r

In [ ]:
loss_function = nn.MSELoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for e in tqdm(range(100)):
    i = 0
    avg_loss = []
    for x,y in dataloader:
        optimizer.zero_grad()

        x = torch.unsqueeze(x, 0).permute(1,2,0)
        # forward
        predictions = model(x)
        
        # loss
        loss = loss_function(predictions, y)
        
        # backward
        loss.backward()

        # optimization
        optimizer.step()
        avg_loss.append(loss.detach().numpy())

        i+=1
        
    if e%5==0:
        avg_loss = np.array(avg_loss)
        print(avg_loss.mean())

In [ ]:
d = random.randint(0,1000)
t = torch.tensor(np.arange(d,d+50)).type(torch.Tensor).view(1,-1,1)
r = model(t).view(-1)

In [ ]:
fig = plt.figure(figsize=(16,4))
plt_x = np.arange(0,t.shape[1])
plt_y = np.arange(d,d+50)

plt_xp = np.arange(5, t.shape[1]+5)
plt_yp = r.detach().numpy()
for i in range(len(r)):
    plt.scatter(plt_x, plt_y, label="real")
    plt.scatter(plt_xp, plt_yp, label="predicted")
    
plt.show()

<a id="2"></a>
# **<div style="padding:10px;color:white;display:fill;border-radius:5px;background-color:rgb(31, 103, 211);font-size:120%;font-family:Verdana;"><center><span> Transformers </span></center></div>**

<aid="2.3"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <p style="color:rgb(172, 28, 44);">Los transformadores, un gran avance en el campo del procesamiento del lenguaje natural, también adoptan varios tipos de arquitecturas de entrada y salida, incluida la configuración de muchos a muchos. En este contexto, los transformadores aportan un enfoque único a la mesa, que contrasta con los métodos utilizados en las redes neuronales recurrentes (RNN) tradicionales como LSTM.</p>
     <p>En una arquitectura de transformador de muchos a muchos, el modelo acepta una secuencia de entradas y devuelve una secuencia de salidas. Sin embargo, a diferencia de los RNN, que procesan secuencias en pasos de tiempo, los transformadores procesan todas las entradas simultáneamente. Esto es posible gracias al mecanismo de atención, que permite que el modelo se centre en diferentes partes de la secuencia de entrada para cada salida, esencialmente creando un "atajo" entre cada entrada y salida.</p>
     <p>Esta arquitectura es especialmente útil en tareas como la traducción automática, donde el modelo necesita comprender el contexto de la oración completa para traducirla con precisión. Del mismo modo, se puede utilizar en tareas como resúmenes de texto o respuesta a preguntas, donde comprender todo el contexto a la vez puede conducir a mejores resultados.</p>
     <p>La arquitectura Many-to-Many de Transformers, combinada con su mecanismo de atención, ofrece un enfoque innovador para abordar tareas secuenciales, lo que convierte a Transformers en una poderosa herramienta en el campo del aprendizaje automático y la inteligencia artificial.</p>
</div>


![image.png](https://images.deepai.org/converted-papers/2001.08317/x1.png)

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, seq_len=50, future=50,  max_len=1000):
        super(CustomDataset).__init__()
        
        self.vocab = {'SOS':1001, 'EOS':1002}
        self.datalist = np.arange(0,max_len)
        self.data, self.targets = self.timeseries(self.datalist, seq_len, future)
        
    def __len__(self):
        #this len will decide the index range in getitem
        return len(self.targets)
    
    def timeseries(self, data, window, future):
        temp = []
        targ = []
        
        for i in range(len(data)-window):
            temp.append(data[i:i+window])
            
        for i in range(len(data)-window -future):
            targ.append(data[i+future:i+window+future])

        return np.array(temp), targ
    
    def __getitem__(self, index):
        x = torch.tensor(self.data[index]).type(torch.Tensor)
        x = torch.cat((torch.tensor([self.vocab['SOS']]), x, torch.tensor([self.vocab['EOS']]))).type(torch.LongTensor)
        
        y = torch.tensor(self.targets[index]).type(torch.Tensor)
        y = torch.cat((torch.tensor([self.vocab['SOS']]), y, torch.tensor([self.vocab['EOS']]))).type(torch.LongTensor)
        
        return x,y
    
dataset = CustomDataset(seq_len=48, future=5, max_len=1000)

In [ ]:
for x, y in dataset:
    print(x)
    print(y)
    break

In [ ]:
dataloader = DataLoader(dataset, batch_size=8, shuffle=True, num_workers=4)
#collate_fn=custom_collector

In [ ]:
for x, y in dataloader:
    print(x.shape)
    print(y.shape)
    break

<a id="3.1"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h3 style="color:rgb(172, 28, 44);">El poder del enmascaramiento y la eficiencia en los transformadores</h3>
     <p>Una de las características notables de Transformers es el uso de máscaras durante el proceso de entrenamiento. El enmascaramiento es un aspecto esencial de la arquitectura de Transformer que evita que el modelo vea tokens futuros en la secuencia de entrada durante el entrenamiento, preservando así la naturaleza secuencial del lenguaje.</p>
     <p>En una tarea como la traducción de idiomas, donde la secuencia de entrada se introduce en el modelo de una sola vez, es fundamental que la predicción de cada palabra no dependa de las palabras que le siguen en la secuencia. Esto se logra aplicando una máscara a la entrada que efectivamente oculta las palabras futuras del modelo durante la fase de entrenamiento.</p>
     <p>El enmascaramiento no solo mantiene la integridad secuencial del lenguaje, sino que también permite que los Transformers entrenen de manera más eficiente que sus contrapartes RNN, como LSTM. A diferencia de los RNN, que procesan secuencias paso a paso y, por lo tanto, requieren tiempos de entrenamiento más largos para secuencias largas, los transformadores pueden procesar todos los tokens de la secuencia simultáneamente, gracias a su mecanismo de atención. Este procesamiento paralelo acelera significativamente el proceso de entrenamiento y permite que el modelo maneje secuencias más largas de manera más efectiva.</p>
     <p>Por lo tanto, mediante el uso de enmascaramiento y su arquitectura única, los Transformers logran superar algunas de las limitaciones de los RNN tradicionales, ofreciendo un enfoque más eficiente y eficaz para las tareas basadas en secuencias en el aprendizaje automático y la inteligencia artificial.</p>
</div>

In [ ]:
class Transformer(nn.Module):
    def __init__(self, num_tokens, dim_model, num_heads, num_layers, input_seq):
        super().__init__()
        self.input_seq = input_seq
        self.num_layers = num_layers
        self.embedding = nn.Embedding(num_tokens, dim_model)
        self.transformer = nn.Transformer(d_model=dim_model, nhead=num_heads,  
                                          num_encoder_layers=3, num_decoder_layers=3, 
                                          dim_feedforward=256, batch_first=True)
        
        self.fc = nn.Linear(dim_model, num_tokens)

    def forward(self, src, tgt, tf=True):
        mask = self.get_mask(tgt.shape[1], teacher_force=tf)
        src = self.embedding(src) 
        tgt = self.embedding(tgt)
        
        out = self.transformer(src, tgt, tgt_mask=mask)
        feed = self.fc(out)
        feed = torch.squeeze(feed,2)
        
        return feed
            
            
    def get_mask(self, size, teacher_force=True):
        if teacher_force:
            mask = torch.tril(torch.ones(size, size) == 1) # Lower triangular matrix
            mask = mask.float()
            mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
            mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

            # EX for size=5:
            # [[0., -inf, -inf, -inf, -inf],
            #  [0.,   0., -inf, -inf, -inf],
            #  [0.,   0.,   0., -inf, -inf],
            #  [0.,   0.,   0.,   0., -inf],
            #  [0.,   0.,   0.,   0.,   0.]]

            return mask
        else:
            mask = torch.tril(torch.zeros(size, size) == 1) # Lower triangular matrix
            mask = mask.float()
            mask = mask.masked_fill(mask == 0, float('-inf')) # Convert zeros to -inf
            mask = mask.masked_fill(mask == 1, float(0.0)) # Convert ones to 0

            return mask

In [ ]:
model = Transformer(num_tokens=1000+3, dim_model=32, num_heads=2, num_layers=2, input_seq=50)

In [ ]:
x.shape, y.shape

In [ ]:
model(x, y).shape 

In [ ]:
t = model(x,y)
t.shape

In [ ]:
t.permute(0,2,1).shape

In [ ]:
loss_function = nn.CrossEntropyLoss()
learning_rate = 1e-3
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

In [ ]:
for e in tqdm(range(25)):
    i = 0
    avg_loss = []
    for x,y in dataloader:
        optimizer.zero_grad()
        
        #one step behind input and output // Like language modeling 
        y_input = y[:, :-1]         # from starting to -1 position
        y_expected = y[:, 1:]       # from 1st position to last 
        # this is done so that in prediction we see a start of token 
        
        # forward
        predictions = model(x, y_input)
        pred = predictions.permute(0, 2, 1)
        
        # loss
        loss = loss_function(pred, y_expected)
        
        # backward
        loss.backward()

        # optimization
        optimizer.step()
        avg_loss.append(loss.detach().numpy())

        i+=1
        
    if e%5==0:
        avg_loss = np.array(avg_loss)
        print(avg_loss.mean())

In [ ]:
torch.squeeze(predictions.topk(1).indices, 2)

In [ ]:
y_expected 

In [ ]:
torch.argmax(pred, dim=1)

<a id="3.2"></a>
<div style="background-color: #f2f2f2; padding: 20px; border-radius: 10px; color: #333; font-family: Arial, sans-serif;">
     <h3 style="color:rgb(172, 28, 44);">El papel de los tokens SOS y EOS en los transformadores</h3>
     <p>En el dominio del procesamiento del lenguaje natural, especialmente cuando se trabaja con modelos de Transformer, los tokens especiales como el inicio de oración (SOS) y el final de oración (EOS) desempeñan un papel fundamental. Estos tokens brindan pistas valiosas sobre los límites de las oraciones, lo que facilita la comprensión de la estructura del lenguaje por parte del modelo.</p>
     <p>El token SOS se agrega al comienzo de cada oración, marcando su inicio. De manera similar, el token EOS se agrega al final de cada oración para indicar su conclusión. Estos tokens sirven como marcadores consistentes que ayudan al modelo a identificar y procesar oraciones como unidades distintas dentro de cuerpos de texto más grandes.</p>
     <p>Además, en el contexto de las tareas de generación de secuencias, estos tokens juegan un papel esencial para determinar cuándo comenzar y finalizar el proceso de generación. Por ejemplo, durante la generación de texto, un token EOS le indica al modelo que debe dejar de generar más tokens.</p>
     <p>Por lo tanto, los tokens SOS y EOS son más que simples marcadores; son componentes integrales en el diseño y funcionamiento de los modelos de Transformer, lo que contribuye significativamente a su capacidad para comprender y generar lenguaje humano de manera efectiva.</p>
</div>

In [ ]:
def predict(model, input_sequence, max_length=50, SOS_token=1000+1, EOS_token=1000+2):
    model.eval()
    
    input_sequence = torch.tensor(input_sequence)
    input_sequence = torch.cat((torch.tensor([SOS_token]), input_sequence, torch.tensor([EOS_token]))).type(torch.LongTensor) 
    input_sequence = torch.unsqueeze(input_sequence,0)
    
    y_input = torch.tensor([1001], dtype=torch.long)
    y_input = torch.unsqueeze(y_input,0)

    for _ in range(max_length):
        
        predictions = model(input_sequence, y_input)
        
        top = predictions.topk(1).indices
        top = torch.squeeze(top, 2)
        
        next_item = torch.unsqueeze(top[:,-1],0)
        y_input = torch.cat((y_input, next_item), dim=1)
        mask = model.get_mask(y_input.shape[1])
        if next_item == EOS_token:
            break

    return y_input.view(-1).tolist()

In [ ]:
d = random.randint(0,900)
t = torch.tensor(np.arange(d,d+48)).type(torch.Tensor)
input_sequence = t
print(t)

In [ ]:
r=predict(model, input_sequence)
print(r)

In [ ]:
fig = plt.figure(figsize=(16,4))

plt_x = np.arange(0,t.shape[0])
plt_y = t

plt_xp = np.arange(5, t.shape[0]+5)
plt_yp = r[1:-2]

plt.scatter(plt_x, plt_y, s=14, color='r', label="real")
plt.scatter(plt_xp, plt_yp, s=7, color='b', label="predicted")
    
plt.legend()
plt.show()